In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import gym

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# Gym Environment

In [12]:
import caviar_tools
from beamselect_env import BeamSelectionEnv

### Hyper Params

In [15]:
epi = [0,10] #[start,end]
gym_env = BeamSelectionEnv(epi)
n_steps = caviar_tools.linecount(epi)

Observation Space : X,Y,Z,pkts_dropped,pkts_transmitted,pkts_buffered,bit_rate

Action Space : [3,64] -> [UE,Possible beams]

In [26]:
gym_env.observation_space

Box([-500. -500. -500.    0.    0.    0.    0.], [5.e+02 5.e+02 5.e+02 1.e+03 1.e+03 2.e+04 1.e+09], (7,), float32)

In [33]:
state,reward,done,info=gym_env.step([1,4])

In [3]:
state

NameError: name 'state' is not defined